In [1]:
import os

In [2]:
%pwd

'c:\\Users\\shwet\\OneDrive\\Desktop\\text-summarization\\research'

In [3]:
os.chdir("../")#research me the text summarization me jana tha isliye

In [4]:
%pwd

'c:\\Users\\shwet\\OneDrive\\Desktop\\text-summarization'

In [ ]:
from dataclasses import dataclass #dataclass: A decorator from the dataclasses module that automatically adds special methods to classes like __init__(), __repr__(), etc.
from pathlib import Path #Comes from the pathlib module. It provides an object-oriented interface to work with file system paths.

@dataclass(frozen=True)
class DataIngestionConfig: #This defines a class to hold configuration settings for the data ingestion step of a machine learning pipeline.
    #Attributes of the Class

    root_dir: Path#The root directory where all data-related operations will be handled.
    source_URL: str#The URL from where the raw data will be downloaded.
    local_data_file: Path#The local file path where the downloaded data will be stored
    unzip_dir: Path#The path where the data will be extracted/unzipped after downloading.

    #basically this is return type


'''@dataclass(frozen=True)....This converts the class into an immutable dataclass, meaning:

You cannot change the attributes after creating an object.

Helps ensure the configuration stays fixed throughout execution (useful in ML pipelines).'''


'@dataclass(frozen=True)....This converts the class into an immutable dataclass, meaning:\n\nYou cannot change the attributes after creating an object.\n\nHelps ensure the configuration stays fixed throughout execution (useful in ML pipelines).'

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])
        

        #DataIngestionConfig is return type
        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config. unzip_dir

        )
        return data_ingestion_config


In [10]:
#update components

import os
import urllib.request as request #to download data from url
import zipfile #TO unzip data
from textSummarizer.logging import logger #custom logger
from textSummarizer.utils.common import get_size

In [11]:
# abhi tk data ingestion config bnaya tha ki kaise khan download hoga ab dataingestion bna rhe jisme actually download hoga

class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config= config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")

        else:
            logger.info(f"file already exists of size:{ get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        ''' 
        zip_file_path: str
        extracts the zip file into the data directory
        function returns None
        
        '''

        unzip_path =self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path) 


In [14]:
#creating pipeline

try:
    config=ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-02 20:58:39,796:INFO:common:yaml file:config\config.yaml loaded successfully]
[2025-08-02 20:58:39,799:INFO:common:yaml file:params.yaml loaded successfully]
[2025-08-02 20:58:39,801:INFO:common:created directory at:artifacts]
[2025-08-02 20:58:39,802:INFO:common:created directory at:artifacts/data_ingestion]


[2025-08-02 20:58:45,722:INFO:2623531039:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1B8C:3ECBE4:18DFFA:3DCF15:688E2EA7
Accept-Ranges: bytes
Date: Sat, 02 Aug 2025 15:28:44 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100024-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1754148521.061568,VS0,VE3725
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: a0d296f25c92c73fa15d273432a055963be5788c
Expires: Sat, 02 Aug 2025 15:33:44 GMT
Source-Age: 0

]
